In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import json

def get_article_links(url, more_clicks=5):
    """
    네이버 뉴스 페이지에서 모든 하위 링크를 수집하는 함수

    Args:
        url (str): 네이버 뉴스 페이지 URL
        more_clicks (int): "기사 더보기" 버튼을 클릭할 횟수

    Returns:
        list: 뉴스 기사 링크 목록
    """

    article_links = []
    # 세션 생성
    session = requests.Session()

    # 페이지 로드
    response = session.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # "더보기" 버튼 클릭
    for _ in range(more_clicks):
        try:
            # 더보기 버튼 찾기
            more_button = soup.find("a", class_="section_more_inner _CONTENT_LIST_LOAD_MORE_BUTTON")

            # 더보기 버튼 클릭을 위한 URL 추출
            more_url = more_button['data-api']

            time.sleep(1)  # 짧은 지연 시간 추가

            # 더보기 버튼 클릭 (API 호출)
            headers = {
                'referer': url,  # referer 헤더 추가
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
            }
            response = session.get(more_url, headers=headers)

            # JSON 데이터 파싱
            json_data = json.loads(response.text)

            # 새로운 기사 목록 HTML 파싱
            new_soup = BeautifulSoup(json_data['html'], "html.parser")

            # 기존 기사 목록에 새로운 기사 목록 추가
            soup.find("ul", class_="type06_headline").append(new_soup.find("ul", class_="type06_headline"))

        except AttributeError:
            print("더보기 버튼을 찾을 수 없습니다.")
            break

    # 뉴스 기사 링크 추출
    articles = soup.select("ul.type06_headline li a.nclicks('RBP.cm')")
    article_links = [article['href'] for article in articles]

    return article_links


# 네이버 뉴스 URL
url = "https://news.naver.com/section/100"

# 모든 하위 링크 수집
article_links = get_article_links(url)

# 결과 출력
print(f"수집된 링크 수: {len(article_links)}")
print(article_links)